# import

In [ ]:
import os
import re
import pandas as pd
import numpy as np

# Google Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/bert_classification_kor/preprocessing/'
os.chdir(path)

Mounted at /content/drive


In [ ]:
os.getcwd()

'/content/drive/MyDrive/bert_classification_kor/preprocessing'

In [ ]:
os.listdir()

['data',
 'crawling_data',
 'Merge_Regex.ipynb',
 'relabelling_data',
 'Untitled0.ipynb']

In [ ]:
path = '/content/drive/MyDrive/bert_classification_kor/preprocessing/relabelling_data/'
filenames = os.listdir(path)

filenames = list(set(filenames))
print(filenames)

['sampledata_이준우.txt', 'sampledata_이한얼.txt', 'sampledata_설진영.txt', 'sampledata_유수현.txt', 'sampledata_길고은.txt', 'sampledata_이신강.txt']


In [ ]:
yogiyo_review = None

for filename in filenames:
    if filename[-3:] == 'csv' :
        temp = pd.read_csv(path + filename, header = None) # sep=','

    elif filename[-3:] == 'txt' :
        temp = pd.read_table(path + filename, header = None) # sep='\t'
    
    if yogiyo_review is None:
        yogiyo_review = temp
        continue

    yogiyo_review = pd.concat([yogiyo_review, temp])

# 인덱스 초기화
yogiyo_review = yogiyo_review.reset_index()

# 만약 자동으로 생성되었던 인덱스 열이 있다면 날려버리기
if 'Unnamed: 0' in yogiyo_review.columns :
    yogiyo_review = yogiyo_review.drop(['Unnamed: 0'], axis=1)
if 'Unnamed: 0.1' in yogiyo_review.columns :
    yogiyo_review = yogiyo_review.drop(['Unnamed: 0.1'], axis=1)
if 'index' in yogiyo_review.columns :
    yogiyo_review = yogiyo_review.drop(['index'], axis=1)


yogiyo_review_df = yogiyo_review.copy()
yogiyo_review_df.columns = ['Total', 'Review']
yogiyo_review_df

,Total,Review
0,1.0,예상시간 분에 아슬아슬하게 맞춰 배달 도착 감튀 다 식어있고 햄버거도 눅눅 실망이 ...
1,3.0,배달짱 빨라쇼아쉬운점은 와퍼가 완전차가와요
2,2.0,문자로 알려주고 문앞에 놓고 가라고 했는데 전화 하시고 나갔는데 계속 앞에서 기다리...
3,1.0,맛이 별로에요 감자 쩐내 냄새나요
4,4.0,무난하게 먹기좋음 굿굿
...,...,...
12846,1.0,햄버거가 다 식어서왔네요포테이토도마찬가지고받자마자 기가막혀서너무하지않나요
12847,5.0,여기는 암만봐도 최고입니다 수고하세요
12848,5.0,잘먹었어요 자주시켜먹어요
12849,2.0,햄버거는 괜찮았고 음료를 아이스 아메리카노로 바꿨는데 원두가 오래된 것 인지 쓴맛과...


In [ ]:
# 불용어 및 특수문자를 없애는 함수
def regex_review(review):
    p = re.compile('[^가-힣 ]')
    result = p.sub('',review)
    result = result.strip()
    result = result.replace('  ', ' ')
    result = result.replace('   ', ' ')
    return result
yogiyo_review_df['regex_review'] = yogiyo_review_df['Review'].apply(regex_review)
yogiyo_review_df

,Total,Review,regex_review
0,1.0,예상시간 분에 아슬아슬하게 맞춰 배달 도착 감튀 다 식어있고 햄버거도 눅눅 실망이 ...,예상시간 분에 아슬아슬하게 맞춰 배달 도착 감튀 다 식어있고 햄버거도 눅눅 실망이 ...
1,3.0,배달짱 빨라쇼아쉬운점은 와퍼가 완전차가와요,배달짱 빨라쇼아쉬운점은 와퍼가 완전차가와요
2,2.0,문자로 알려주고 문앞에 놓고 가라고 했는데 전화 하시고 나갔는데 계속 앞에서 기다리...,문자로 알려주고 문앞에 놓고 가라고 했는데 전화 하시고 나갔는데 계속 앞에서 기다리...
3,1.0,맛이 별로에요 감자 쩐내 냄새나요,맛이 별로에요 감자 쩐내 냄새나요
4,4.0,무난하게 먹기좋음 굿굿,무난하게 먹기좋음 굿굿
...,...,...,...
12846,1.0,햄버거가 다 식어서왔네요포테이토도마찬가지고받자마자 기가막혀서너무하지않나요,햄버거가 다 식어서왔네요포테이토도마찬가지고받자마자 기가막혀서너무하지않나요
12847,5.0,여기는 암만봐도 최고입니다 수고하세요,여기는 암만봐도 최고입니다 수고하세요
12848,5.0,잘먹었어요 자주시켜먹어요,잘먹었어요 자주시켜먹어요
12849,2.0,햄버거는 괜찮았고 음료를 아이스 아메리카노로 바꿨는데 원두가 오래된 것 인지 쓴맛과...,햄버거는 괜찮았고 음료를 아이스 아메리카노로 바꿨는데 원두가 오래된 것 인지 쓴맛과...


In [ ]:
# 정제한 리뷰의 길이를 센 컬럼 추가
yogiyo_review_df['길이'] = [len(regex_review) for regex_review in yogiyo_review_df.regex_review.to_list()]
yogiyo_review_df

,Total,Review,regex_review,길이
0,1.0,예상시간 분에 아슬아슬하게 맞춰 배달 도착 감튀 다 식어있고 햄버거도 눅눅 실망이 ...,예상시간 분에 아슬아슬하게 맞춰 배달 도착 감튀 다 식어있고 햄버거도 눅눅 실망이 ...,57
1,3.0,배달짱 빨라쇼아쉬운점은 와퍼가 완전차가와요,배달짱 빨라쇼아쉬운점은 와퍼가 완전차가와요,23
2,2.0,문자로 알려주고 문앞에 놓고 가라고 했는데 전화 하시고 나갔는데 계속 앞에서 기다리...,문자로 알려주고 문앞에 놓고 가라고 했는데 전화 하시고 나갔는데 계속 앞에서 기다리...,90
3,1.0,맛이 별로에요 감자 쩐내 냄새나요,맛이 별로에요 감자 쩐내 냄새나요,18
4,4.0,무난하게 먹기좋음 굿굿,무난하게 먹기좋음 굿굿,12
...,...,...,...,...
12846,1.0,햄버거가 다 식어서왔네요포테이토도마찬가지고받자마자 기가막혀서너무하지않나요,햄버거가 다 식어서왔네요포테이토도마찬가지고받자마자 기가막혀서너무하지않나요,40
12847,5.0,여기는 암만봐도 최고입니다 수고하세요,여기는 암만봐도 최고입니다 수고하세요,20
12848,5.0,잘먹었어요 자주시켜먹어요,잘먹었어요 자주시켜먹어요,13
12849,2.0,햄버거는 괜찮았고 음료를 아이스 아메리카노로 바꿨는데 원두가 오래된 것 인지 쓴맛과...,햄버거는 괜찮았고 음료를 아이스 아메리카노로 바꿨는데 원두가 오래된 것 인지 쓴맛과...,58


In [ ]:
# 글자 수가 12개 이상, 80개 이하인 것만 남기기
# 너무 길어도 학습이 잘 안 되기 때문에
yogiyo_review_df = yogiyo_review_df[(yogiyo_review_df['길이'] >= 12) & (yogiyo_review_df['길이'] <= 80)]

# 길이 열을 기준으로 내림차순으로 정렬
print(yogiyo_review_df.sort_values(by='길이', axis= 0, ascending=False))

       Total  ...  길이
1329     1.0  ...  80
6961     1.0  ...  80
7073     1.0  ...  80
7629     1.0  ...  80
5349     5.0  ...  80
...      ...  ...  ..
381      5.0  ...  12
1760     5.0  ...  12
12441    3.0  ...  12
4414     5.0  ...  12
1870     1.0  ...  12

[10112 rows x 4 columns]


In [ ]:
yogiyo_review_df.Total.value_counts()

5.0    3571
1.0    3564
3.0    1565
4.0     727
2.0     685
Name: Total, dtype: int64

In [ ]:
# txt 파일로 저장하기
# 리뷰와 평점열만 저장
# 헤더랑 인덱스 열 저장 안 하기
yogiyo_review_df[['Total', 'regex_review']].to_csv('/content/drive/MyDrive/bert_classification_kor/preprocessing/data/relabeling_data.txt', sep= '\t', header= None, index = None)

# 평점 병합
1점, 2점, 3점, 4점, 5점 -> 1점, 3점, 5점

In [ ]:
yogiyo_review_df.Total.unique()

array([1., 3., 4., 5., 2.])

In [ ]:
def change(x) :
    if x == 2.0 :
        return 3.0
    elif x == 4.0 :
        return 3.0
    else :
        return x

# yogiyo_review_df['Total'] = yogiyo_review_df['Total'].apply(change)
yogiyo_review_df['Total'] = yogiyo_review_df['Total'].apply(lambda x: 3.0 if x == 2.0 else 3.0 if x == 4.0 else x)
yogiyo_review_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Total,Review,regex_review,길이
0,1.0,예상시간 분에 아슬아슬하게 맞춰 배달 도착 감튀 다 식어있고 햄버거도 눅눅 실망이 ...,예상시간 분에 아슬아슬하게 맞춰 배달 도착 감튀 다 식어있고 햄버거도 눅눅 실망이 ...,57
1,3.0,배달짱 빨라쇼아쉬운점은 와퍼가 완전차가와요,배달짱 빨라쇼아쉬운점은 와퍼가 완전차가와요,23
3,1.0,맛이 별로에요 감자 쩐내 냄새나요,맛이 별로에요 감자 쩐내 냄새나요,18
4,3.0,무난하게 먹기좋음 굿굿,무난하게 먹기좋음 굿굿,12
5,3.0,배달이 느렸지만 배달원이 친절했어요 그치만 늦어서 감자가 다 불었더라구요,배달이 느렸지만 배달원이 친절했어요 그치만 늦어서 감자가 다 불었더라구요,40
...,...,...,...,...
12846,1.0,햄버거가 다 식어서왔네요포테이토도마찬가지고받자마자 기가막혀서너무하지않나요,햄버거가 다 식어서왔네요포테이토도마찬가지고받자마자 기가막혀서너무하지않나요,40
12847,5.0,여기는 암만봐도 최고입니다 수고하세요,여기는 암만봐도 최고입니다 수고하세요,20
12848,5.0,잘먹었어요 자주시켜먹어요,잘먹었어요 자주시켜먹어요,13
12849,3.0,햄버거는 괜찮았고 음료를 아이스 아메리카노로 바꿨는데 원두가 오래된 것 인지 쓴맛과...,햄버거는 괜찮았고 음료를 아이스 아메리카노로 바꿨는데 원두가 오래된 것 인지 쓴맛과...,58


In [ ]:
yogiyo_review_df.Total.value_counts()

5.0    3571
1.0    3564
3.0    2977
Name: Total, dtype: int64

# 샘플 데이터 생성

In [ ]:
sample_df =pd.DataFrame(columns = ['Total', 'Review', 'regex_review', '길이'])

# 3점이 제일 적기에 
for rating in yogiyo_review_df['Total'].unique() :
    temporary_df = yogiyo_review_df.query('Total==@rating').sample(n=2977)
    sample_df = pd.concat([sample_df, temporary_df])
    
sample_df = sample_df.reset_index(drop=True)

sample_df = sample_df.sort_values(by='Total', axis= 0)

sample_df = sample_df[['Total', 'regex_review']]
sample_df

,Total,regex_review
0,1.0,맛은 실망 음료수가 이게 뭔지 차라리 주지나 말지
1979,1.0,치킨에서 냄새도 나고 배달도 늦고 배달원도 불친절하고
1980,1.0,기름찌든 맛이랑 닭누린내가 너무 심하네요
1981,1.0,늦게 온 음식 눅눅해진 것도 모자라 라지로 주문한 후라이가 미디엄으로 온 매직
1982,1.0,배달도 잘못주고 가게에서 전화해서 말하는 도중에 끊고 뭐임
...,...,...
6949,5.0,음식맛도 괜찮고요 정말 신속 배달 짱이에욤
6950,5.0,항상 맛있게 잘 먹고있습니다
6951,5.0,배달 시간 최악 차디찬 햄버거 버거는 왜케 또 빈한지 행사라 그런건지 원래 그런건지
6942,5.0,먹다가 생각나서 사진을 급하게 찍었네요 역시 맛있어요


In [ ]:
2977*3

8931

In [ ]:
# txt 파일로 저장하기
# 헤더랑 인덱스 열 저장 안 하기
sample_df.to_csv('/content/drive/MyDrive/bert_classification_kor/preprocessing/data/relabeling_data_sampleling.txt', sep= '\t', header= None, index = None)

# 네이버 맞춤법 검사기로 한번 더 정제
오래 걸림(30분 넘게)<br>
안 하고 싶으면 안 해도 됨

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-05rtua9t
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-05rtua9t
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=72a086b884ad89323af957740ca4fe3547cccbfc4bbba834a570d73373f79826
  Stored in directory: /tmp/pip-ephem-wheel-cache-6wt20xqa/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker

In [ ]:
review_hanspell = []

for review in sample_df["regex_review"] :
    review_checked = spell_checker.check(review) # 하나씩 맞춤법 검사
    review_checked = list(review_checked) # 리스트로 변환
    review_checked = review_checked[2] # list의 인덱스 2에 교정된 문장이 있음 
    review_hanspell.append(review_checked)

review_hanspell[:10]

['맛은 실망 음료수가 이게 뭔지 차라리 주지나 말지',
 '치킨에서 냄새도 나고 배달도 늦고 배달원도 불친절하고',
 '기름 찌든 맛이랑 닭 누린내가 너무 심하네요',
 '늦게 온 음식 눅눅해진 것도 모자라 라지로 주문한 프라이가 미디엄으로 온 매직',
 '배달도 잘못 주고 가게에서 전화해서 말하는 도중에 끊고 뭐임',
 '주문한지 분도 안돼서 배달 와서 의아하고 와이프랑 저랑 지금 배탈로 고생 중입니다',
 '한 입 먹자마자 다들 깜놀 너무 짜서 거의 못 먹고 버렸어요 호식이 좋아해서 친구들이랑 여기저기서 많이 시켜 먹어봤는데 이런 적은 첨이네요',
 '배달이 시간 이상 걸려왔는데 소스가 다 흘러 박스며 가방이며 다 젖어 왔네요 배달의 문제인지 포장의 문제인지 신경 좀 써야 될 것 같아요',
 '저기 다 좋은 게요 아무리 가정집이라지만 인간적으로 빨대 없이 먹으라는 건 아니지 않나요  빨대 안 주는 건 여기뿐인 듯 요',
 '주말이라 바쁘신 건 알겠지만 케첩이랑 빨대 안 주신 거는 좀 난감하네요']

In [ ]:
# 기존의 리뷰에 맞춤법 교정한 리뷰 넣기
sample_df['regex_review'] = review_hanspell

In [ ]:
sample_df

,Total,regex_review
0,1.0,맛은 실망 음료수가 이게 뭔지 차라리 주지나 말지
1979,1.0,치킨에서 냄새도 나고 배달도 늦고 배달원도 불친절하고
1980,1.0,기름 찌든 맛이랑 닭 누린내가 너무 심하네요
1981,1.0,늦게 온 음식 눅눅해진 것도 모자라 라지로 주문한 프라이가 미디엄으로 온 매직
1982,1.0,배달도 잘못 주고 가게에서 전화해서 말하는 도중에 끊고 뭐임
...,...,...
6949,5.0,음식 맛도 괜찮고요 정말 신속 배달 최고에욤
6950,5.0,항상 맛있게 잘 먹고 있습니다
6951,5.0,배달 시간 최악 차디찬 햄버거 버거는 왜 이렇게 또 빈 한지 행사라 그런 건지 원래...
6942,5.0,먹다가 생각나서 사진을 급하게 찍었네요 역시 맛있어요


In [ ]:
# txt 파일로 저장하기
# 헤더랑 인덱스 열 저장 안 하기
sample_df.to_csv('/content/drive/MyDrive/bert_classification_kor/preprocessing/data/relabeling_data_sampleing_hanspell.txt', sep= '\t', header= None, index = None)